In [68]:
# 기본 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
df = pd.read_csv('book.csv', encoding='euc-kr')

In [70]:
df_droped = df.dropna(subset=['Title'])
df_droped = df_droped[df_droped['Title'].str.strip() != '']

In [71]:
df_droped['Price'] = df_droped['Price'].str.replace(',','')

In [72]:
import re

# 영어 제목만
def is_english_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[a-zA-Z0-9 .,\-\'!?]+$', text))

# 영어로만 구성된 책 제목만 가져옴
final_e = df_droped[df_droped['Title'].apply(is_english_title)]

# 한국 제목만
def is_korean_title(text):
    # 영어 알파벳, 공백, 일부 특수문자(예: ',!?.)만 허용
    return bool(re.match(r'^[가-힣0-9 .,\-\'!?]+$', text))

# 한국어로만 구성된 책 제목만 가져옴
final_k = df_droped[df_droped['Title'].apply(is_korean_title)]

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [74]:
tfidf_vectorizer_e= TfidfVectorizer(max_features=20000)
tfidf_vectorizer_k= TfidfVectorizer(max_features=20000)
X_tfidf_e = tfidf_vectorizer_e.fit_transform(final_e)
X_tfidf_k = tfidf_vectorizer_k.fit_transform(final_k)

In [76]:
# 클러스터링 수행
kmeans_e = KMeans(n_clusters=10, random_state=42)
kmeans_k = KMeans(n_clusters=10, random_state=42)
kmeans_e.fit(X_tfidf_e)
kmeans_k.fit(X_tfidf_k)

KMeans(n_clusters=10, random_state=42)

In [77]:
################# Word2Vec  #################
from gensim.models import Word2Vec ,FastText
from sklearn.cluster import KMeans

import pandas as pd
import numpy as np

def vectorize_w_word2vec(text):
    words = text.split()
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)

def vectorize_w_fasttext(text):
    words = text.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(word_vectors, axis=0)

sentences = [title.split() for title in df_k['Title']]

word2vec_model = Word2Vec(sentences, vector_size=64, window=5, min_count=1) # 워드 투 벡터 : 주변 단어들을 학습에 사용 
fasttext_model = FastText(sentences, vector_size=64, window=5, min_count=1) # fast 텍스트 : 서브 워드 이용

title_vectors_k = np.array([vectorize_w_word2vec(title) for title in df_k['Title']])

## 클러스터링

In [83]:
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(title_vectors_k)

# 각 인덱스의 클러스터 할당하기
df_k['Cluster'] = kmeans.labels_

for cluster in range(3): # 클러스터의 수에 따라 범위 조정
    print(f"Cluster {cluster}:")
    print(df_k[df_k['Cluster'] == cluster]['Title'].head(3), '\n') # 각 클러스터에 속한 책 제목 출력

Cluster 0:
409    묻고 답하는 한국사카페 1
442        두근두근 중국어 1
476         런웨이의 연인 1
Name: Title, dtype: object 

Cluster 1:
94      건축견적이야기 3 
467           마기 3
469    주문배달의 왕자님 3
Name: Title, dtype: object 

Cluster 2:
1024           한국사 편지 세트
1724      설민석의 한국사 대모험 5
1819    특종! 70명으로 읽는 한국사
Name: Title, dtype: object 



C:\Users\crazy\AppData\Local\Temp\ipykernel_7376\3850645773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_k['Cluster'] = kmeans.labels_


In [145]:
print("Word2Vec의 결과")
for word, sim in word2vec_model.wv.most_similar('투자'):
    print(f"Word: {word:<5}, similarity: {sim:.2f}")

print("\nFastText의 결과")
for word, sim in fasttext_model.wv.most_similar('투자'):
    print(f"Word: {word:<5}, similarity: {sim:.2f}")


Word2Vec의 결과
Word: 털털털  , similarity: 0.50
Word: 전산회계 , similarity: 0.47
Word: 이대로가 , similarity: 0.45
Word: 알리는  , similarity: 0.44
Word: 마흔   , similarity: 0.44
Word: 책    , similarity: 0.43
Word: 그들에게 , similarity: 0.43
Word: 카메라  , similarity: 0.43
Word: 알았더라면, similarity: 0.42
Word: 애장판  , similarity: 0.41

FastText의 결과
Word: 에밀   , similarity: 0.46
Word: 시대가  , similarity: 0.45
Word: 사들인  , similarity: 0.44
Word: 핵을   , similarity: 0.43
Word: 너한테도 , similarity: 0.43
Word: 10주년 , similarity: 0.43
Word: 확대방안 , similarity: 0.42
Word: 도전,  , similarity: 0.42
Word: 78층  , similarity: 0.42
Word: 절반쯤  , similarity: 0.41
